# Обучение

In [22]:
import cv2
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from collections import Counter

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as T
import torchvision.models.video as models

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.express as px
import plotly.io as pio

from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# директория со всеми промежуточными датасетами
DATASETS_PATH = '/content/drive/MyDrive/диплом/Datasets'
# размеченные видео по классам элементов
ELEMENTS_PATH = r'/content/drive/MyDrive/диплом/По элементам'

In [5]:
# датасет: путь до видео, лейбл (название элемента)

df_elements = pd.DataFrame()
BD_path, BD_names = [],[]
for group in os.listdir(ELEMENTS_PATH):  # равновесия, прыжки, повороты
  df = pd.DataFrame()
  for elem_name in os.listdir(os.path.join(ELEMENTS_PATH,group)):  # название классов, лейблы
    for video_name in os.listdir(os.path.join(ELEMENTS_PATH,group,elem_name)):  # название видео
      BD_path.append(os.path.join(ELEMENTS_PATH,group,elem_name, video_name))
      BD_names.append(f'{group}_{elem_name}') # название класса

df_elements['BD_path'] = BD_path
df_elements['BD_names'] = BD_names
print(df_elements.shape)
df_elements.head(3)

(3311, 2)


,BD_path,BD_names
0,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе
1,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе
2,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе


In [6]:
# # Преобразуем классы в индексы, присвоив порядковый номер класса через LabelEncoder

le = LabelEncoder()
df_elements['BD_label'] = le.fit_transform( df_elements['BD_names'])
df_elements.head(3)

,BD_path,BD_names,BD_label
0,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе,15
1,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе,15
2,/content/drive/MyDrive/диплом/По элементам/Пов...,Повороты_Пассе,15


# Подготовка датасета

In [7]:
df = pd.read_excel(os.path.join(DATASETS_PATH, 'Датасет в модель.xlsx'))
df.head(5)

,BD_path,BD_label,video_name
0,/content/drive/MyDrive/диплом/Video silhouette...,15,поворот пассе — сделано в Clipchamp_0.mp4
1,/content/drive/MyDrive/диплом/Video silhouette...,15,Диана Чугунихина поворт пассе — сделано в Clip...
2,/content/drive/MyDrive/диплом/Video silhouette...,15,Диана Чугунихина поворт пассе — сделано в Clip...
3,/content/drive/MyDrive/диплом/Video silhouette...,15,Диана Чугунихина поворт пассе — сделано в Clip...
4,/content/drive/MyDrive/диплом/Video silhouette...,15,Диана Чугунихина поворт пассе — сделано в Clip...


In [8]:
df.shape

(6232, 3)

In [9]:
pd.set_option('display.max_colwidth', None)
# df['BD_path'] = df['BD_path'].str.replace('/content/drive/MyDrive/диплом', '/content/drive/MyDrive/пм21_финашка/диплом/диплом')
# df['BD_path']

## Train/test

In [10]:
# разделение на тренировочную и тестовую выборку в соотношении 0.8/0.2 соотвественно
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

train_df.shape, test_df.shape

((4986, 3), (1246, 3))

In [11]:
# составляем датасет для подачи видео в модели классификации

class VideoDataset(Dataset):

  # инициализация
  def __init__(self, df, transform = None):
    self.df = df.reset_index(drop = False)
    self.transform = transform

  # размер, длина датасета
  def __len__(self):
    return len(self.df)

  # чтение видео для подачи в модель
  def read_video(self, video_path):

    cap = cv2.VideoCapture(video_path)  # чтение видео
    frames = []  # все кадры
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (112, 112))  # ресайз для обучения модели
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # в ргб
        frames.append(frame)
    cap.release()

    # конвертируем в тензор (T x C x H x W)
    frames = np.array(frames).astype(np.float32) / 255.0
    frames = torch.from_numpy(frames).permute(0, 3, 1, 2)  # T C H W, время, каналы, высота, ширина
    return frames

  def __getitem__(self, idx):  # срез по индексу
    video_path = self.df.loc[idx, 'BD_path']
    label = self.df.loc[idx, 'BD_label']

    video = self.read_video(video_path)  # обработка кадров видео
    if self.transform:
        video = self.transform(video)
    return video, label


In [12]:
train_dataset = VideoDataset(train_df)
test_dataset = VideoDataset(test_df)
train_dataset

In [13]:
# учитываем неравномерность распределения классов
def get_sampler(labels):
  label_counts = Counter(labels)  # количество раз, которое встречается каждый класс
  weights = [1 / label_counts[label] for label in labels]  # вес каждого класса, обратное число частоте классе
  return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

In [14]:
# все видеозаписи имеют разную длительность, обрезать видео нельзя, так как может быть вырезан важный фрагмент элемента
# выравниваем разную длительность до максимальной паддингом, заполняем нулями
def collate_fn(batch):
    # паддинг до максимальной длины в батче
    max_len = max([video.shape[0] for video, _ in batch])  # vмакс длительность видео в батче
    padded_videos = []
    labels = []
    for video, label in batch:
        pad_len = max_len - video.shape[0]  # оставшееся время до макс длительности
        if pad_len > 0:
            pad = torch.zeros((pad_len, 3, 112, 112))  # заполняем нулями
            video = torch.cat([video, pad], dim=0)   # конкатим исхолдное и дополненное черное видео
        padded_videos.append(video)
        labels.append(label)
    videos = torch.stack(padded_videos).permute(0, 2, 1, 3, 4)  # B C T H W
    labels = torch.tensor(labels)  # в тензор
    return videos, labels

In [15]:
sampler = get_sampler(train_df['BD_label'].tolist())  # веса классов
train_loader = DataLoader(train_dataset, batch_size=4, sampler=sampler, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)
train_loader

# Models

In [16]:
class VideoClassifier(nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.resnet = models.r3d_18(pretrained = True)
    self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

  def forward(self, x) : # прямой проход через модель
    return self.resnet(x)


In [17]:
def train(model, train_loader, test_loader, num_epochs=5, device='cuda'):
    model.to(device)  # переносим все вычисления на гпу
    criterion = nn.CrossEntropyLoss()  # функция потерь
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # модифицированный стохастический градиентный спуск

    train_losses = []  # потери на трейне
    test_losses = []   # потери на тесте
    train_accuracies = []  # точность на трейне
    test_accuracies = []  # точность на тесте

    for epoch in range(num_epochs):   # эпохи обучения
        model.train()   # в режим обучения
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        all_train_preds = []
        all_train_labels = []

        for videos, labels in tqdm(train_loader): # обучение на трейне
            videos, labels = videos.to(device), labels.to(device)  # переносим вычисления на гпу
            outputs = model(videos)  # прогоняем через модель
            loss = criterion(outputs, labels)  # рассчитываем функцию потерь

            optimizer.zero_grad()  # расчет градиента
            loss.backward()   # обратное распространение ошибки
            optimizer.step()

            train_loss += loss.item()

            # Accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # предсказанные предикты и реальнын метки для подсчета метрик
            all_train_preds.extend(predicted.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())

        train_losses.append(train_loss / len(train_loader))  # Средняя потеря для эпохи
        train_accuracies.append(correct_train / total_train)  # Точность для эпохи

        #  Precision, Recall,  F1 на трейне
        train_precision = precision_score(all_train_labels, all_train_preds, average='macro')
        train_recall = recall_score(all_train_labels, all_train_preds, average='macro')
        train_f1 = f1_score(all_train_labels, all_train_preds, average='macro')

        # Тестирование
        model.eval()
        test_loss = 0.0
        correct_test = 0
        total_test = 0
        all_test_preds = []
        all_test_labels = []
        with torch.no_grad():
            for videos, labels in tqdm(test_loader):
                videos, labels = videos.to(device), labels.to(device)  # переносим вычисления на гпу
                outputs = model(videos)  # предсказываем
                loss = criterion(outputs, labels)  # рассчитываем функцию потерь
                test_loss += loss.item()

                # Accuracy
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

                # сохраняем предикты и реальные метки для подсчета метрик
                all_test_preds.extend(predicted.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())

        test_losses.append(test_loss / len(test_loader))  # Средняя потеря для эпохи
        test_accuracies.append(correct_test / total_test)  # Точность для эпохи

        #  Precision, Recall, and F1 на тесте
        test_precision = precision_score(all_test_labels, all_test_preds, average='macro')
        test_recall = recall_score(all_test_labels, all_test_preds, average='macro')
        test_f1 = f1_score(all_test_labels, all_test_preds, average='macro')

        print(f"Epoch {epoch + 1} completed.")
        print(f'Train Loss: {train_losses}, Train Accuracy: {train_accuracies}, '
              f'Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}')
        print(f'Test Loss: {test_losses}, Test Accuracy: {test_accuracies}, '
              f'Test Precision: {test_precision}, Test Recall: {test_recall}, Test F1: {test_f1}')

        # сохраняем веса модели
        model_save_path_cnn = os.path.join(DATASETS_PATH, f"r3d_18_model_weights_{epoch + 1}_epoch.pth")
        torch.save(model.state_dict(), f"r3d_18_model_weights_{epoch + 1}_epoch.pth")  # локально
        torch.save(model.state_dict(), model_save_path_cnn)  # на диск
        print(f"Веса модели сохранены в {model_save_path_cnn}")

    # Рисуем графики потерь и точности
    epochs = range(1, num_epochs + 1)

    plt.figure(figsize=(12, 6))

    # График потерь
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # График точности
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy')
    plt.plot(epochs, test_accuracies, label='Test Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

    return model  # возвращаем модель

In [ ]:
model = VideoClassifier(num_classes = df['BD_label'].nunique())  # загружаем предобученную модель resnet3d
# Обучение
model = train(model, train_loader, test_loader, num_epochs = 7)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 312/312 [16:20<00:00,  3.14s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1 completed.
Train Loss: [3.1901279501945567], Train Accuracy: [0.28259125551544323], Train Precision: 0.28132578280336623, Train Recall: 0.2706714584293034, Train F1: 0.2521998792915857
Test Loss: [1.9244641353000658], Test Accuracy: [0.45585874799357945], Test Precision: 0.48819844813820784, Test Recall: 0.48180700419313494, Test F1: 0.41058740036276775
Веса модели сохранены в /content/drive/MyDrive/диплом/Datasets/r3d_18_model_weights_1_epoch.pth


100%|██████████| 312/312 [05:27<00:00,  1.05s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2 completed.
Train Loss: [3.1901279501945567, 1.388042894631935], Train Accuracy: [0.28259125551544323, 0.7210188527878059], Train Precision: 0.7144517018370715, Train Recall: 0.7157296223930063, Train F1: 0.7049338275065402
Test Loss: [1.9244641353000658, 0.8371357954706087], Test Accuracy: [0.45585874799357945, 0.7495987158908507], Test Precision: 0.7776043690397484, Test Recall: 0.8022310616775604, Test F1: 0.7649325821126088
Веса модели сохранены в /content/drive/MyDrive/диплом/Datasets/r3d_18_model_weights_2_epoch.pth


 85%|████████▍ | 1057/1247 [40:29<07:02,  2.22s/it]

In [ ]:
# Путь для загрузки весов модели
model_save_path_cnn = os.path.join(DATASETS_PATH,  'r3d_18_weights', f"r3d_18_model_weights_{2}_epoch.pth")   # дообучаем сохраненные веса после 2 эпохи
model = VideoClassifier(num_classes=df['BD_label'].nunique())  # создаем модель
model.load_state_dict(torch.load(model_save_path_cnn, map_location=torch.device('cuda')))  # загружаем сохраненнные веса модели после 2 эпохи

model = train(model, train_loader, test_loader, num_epochs=5)  # 7 эпох суммарно будет

100%|██████████| 312/312 [27:02<00:00,  5.20s/it]


Epoch 1 completed.
Train Loss: [0.5836442147591469], Train Accuracy: [0.8882872041716807], Train Precision: 0.8847584383931377, Train Recall: 0.8855590712188672, Train F1: 0.8841402134206447
Test Loss: [0.5920187282971461], Test Accuracy: [0.8154093097913323], Test Precision: 0.8738780525820655, Test Recall: 0.8573657697840946, Test F1: 0.8466886626619102
Веса модели сохранены в /content/drive/MyDrive/кони диплом, нужен гпу/диплом/Datasets/r3d_18_model_weights_1_epoch.pth


100%|██████████| 312/312 [05:27<00:00,  1.05s/it]


Epoch 2 completed.
Train Loss: [0.5836442147591469, 0.30965773154867166], Train Accuracy: [0.8882872041716807, 0.9430405134376254], Train Precision: 0.9418878276708481, Train Recall: 0.9423230744153749, Train F1: 0.9418196416136982
Test Loss: [0.5920187282971461, 0.5009726342723272], Test Accuracy: [0.8154093097913323, 0.8611556982343499], Test Precision: 0.8911664698051203, Test Recall: 0.8879410489601538, Test F1: 0.8746535576586286
Веса модели сохранены в /content/drive/MyDrive/кони диплом, нужен гпу/диплом/Datasets/r3d_18_model_weights_2_epoch.pth


100%|██████████| 312/312 [05:29<00:00,  1.06s/it]


Epoch 3 completed.
Train Loss: [0.5836442147591469, 0.30965773154867166, 0.20509072396987876], Train Accuracy: [0.8882872041716807, 0.9430405134376254, 0.9604893702366627], Train Precision: 0.9602463578060345, Train Recall: 0.9597680092702408, Train F1: 0.9597486178818733
Test Loss: [0.5920187282971461, 0.5009726342723272, 0.49249207803010353], Test Accuracy: [0.8154093097913323, 0.8611556982343499, 0.8579454253611557], Test Precision: 0.8880489416389684, Test Recall: 0.8787218020802605, Test F1: 0.8663040119617564
Веса модели сохранены в /content/drive/MyDrive/кони диплом, нужен гпу/диплом/Datasets/r3d_18_model_weights_3_epoch.pth


100%|██████████| 312/312 [05:28<00:00,  1.05s/it]


Epoch 4 completed.
Train Loss: [0.5836442147591469, 0.30965773154867166, 0.20509072396987876, 0.15121372726633128], Train Accuracy: [0.8882872041716807, 0.9430405134376254, 0.9604893702366627, 0.9715202567188127], Train Precision: 0.9705737016409041, Train Recall: 0.9701580034861348, Train F1: 0.9702248720877289
Test Loss: [0.5920187282971461, 0.5009726342723272, 0.49249207803010353, 0.49388762363265987], Test Accuracy: [0.8154093097913323, 0.8611556982343499, 0.8579454253611557, 0.8707865168539326], Test Precision: 0.8885479028531886, Test Recall: 0.9003900994298727, Test F1: 0.8827546650481477
Веса модели сохранены в /content/drive/MyDrive/кони диплом, нужен гпу/диплом/Datasets/r3d_18_model_weights_4_epoch.pth


 34%|███▎      | 420/1247 [16:19<33:58,  2.46s/it]

In [10]:
epochs = list(range(1, 7))

Train_Losses = [3.1901279501945567, 1.388042894631935, 0.5836442147591469, 0.30965773154867166, 0.20509072396987876, 0.15121372726633128]
Train_Accuracy = [0.28259125551544323, 0.7210188527878059, 0.8882872041716807, 0.9430405134376254,   0.9604893702366627, 0.9715202567188127]
Train_Precision = [0.28132578280336623, 0.7144517018370715, 0.8847584383931377, 0.9418878276708481,  0.9602463578060345, 0.9705737016409041]
Train_Recall = [0.2706714584293034, 0.7157296223930063, 0.8855590712188672, 0.9423230744153749, 0.9597680092702408, 0.9701580034861348]
Train_F1 = [0.2521998792915857, 0.7049338275065402, 0.8841402134206447, 0.9418196416136982,    0.9597486178818733, 0.9702248720877289]

Test_Losses = [1.9244641353000658, 0.8371357954706087, 0.5920187282971461, 0.5009726342723272,   0.49249207803010353, 0.49388762363265987]
Test_Accuracy = [0.45585874799357945, 0.7495987158908507, 0.8154093097913323, 0.8611556982343499, 0.8579454253611557, 0.8707865168539326]
Test_Precision = [0.48819844813820784, 0.7776043690397484, 0.8738780525820655, 0.8911664698051203,0.8880489416389684, 0.8885479028531886]
Test_Recall = [0.48180700419313494, 0.8022310616775604, 0.8573657697840946, 0.8879410489601538,   0.8787218020802605, 0.9003900994298727]
Test_F1 = [0.41058740036276775, 0.7649325821126088, 0.8466886626619102, 0.8746535576586286,  0.8663040119617564, 0.8827546650481477]

fig = sp.make_subplots(rows=5, cols=1, subplot_titles=("График Потерь", "График Точности", 'График Recall',  'График Precision', 'График F1'))

train_line = dict(color='blue')
test_line = dict(color='red')

show_legend_train = True
show_legend_test = True

# График потерь
fig.add_trace(go.Scatter(x=epochs, y=Train_Losses, mode='lines+markers', name='Train Loss',
                         line=train_line, showlegend=show_legend_train), row=1, col=1)
fig.add_trace(go.Scatter(x=epochs, y=Test_Losses, mode='lines+markers', name='Test Loss',
                         line=test_line, showlegend=show_legend_test), row=1, col=1)
# Отключаем отображение легенды для остальных графиков, чтобы она не повторялась
show_legend_train = False
show_legend_test = False

# График точности
fig.add_trace(go.Scatter(x=epochs, y=Train_Accuracy, mode='lines+markers', name='Train Accuracy',
                         line=train_line, showlegend=show_legend_train), row=2, col=1)
fig.add_trace(go.Scatter(x=epochs, y=Test_Accuracy, mode='lines+markers', name='Test Accuracy',
                         line=test_line, showlegend=show_legend_test), row=2, col=1)

# График recall
fig.add_trace(go.Scatter(x=epochs, y=Train_Recall, mode='lines+markers', name='Train Recall',
                         line=train_line, showlegend=show_legend_train), row=3, col=1)
fig.add_trace(go.Scatter(x=epochs, y=Test_Recall, mode='lines+markers', name='Test Recall',
                         line=test_line, showlegend=show_legend_test), row=3, col=1)

# График precision
fig.add_trace(go.Scatter(x=epochs, y=Train_Precision, mode='lines+markers', name='Train Precision',
                         line=train_line, showlegend=show_legend_train), row=4, col=1)
fig.add_trace(go.Scatter(x=epochs, y=Test_Precision, mode='lines+markers', name='Test Precision',
                         line=test_line, showlegend=show_legend_test), row=4, col=1)

# График F1
fig.add_trace(go.Scatter(x=epochs, y=Train_F1, mode='lines+markers', name='Train F1',
                         line=train_line, showlegend=show_legend_train), row=5, col=1)
fig.add_trace(go.Scatter(x=epochs, y=Test_F1, mode='lines+markers', name='Test F1',
                         line=test_line, showlegend=show_legend_test), row=5, col=1)

# Настройка общих заголовков и осей
fig.update_layout(title='Графики метрик на Train и Test',
                  yaxis_title='Значение',
                  showlegend=True,  # Отображение общей легенды
                  plot_bgcolor='black',  # Цвет фона графика
                  paper_bgcolor='black',  # Цвет фона всей области графика
                  font_color='white'  # Цвет шрифта
                  )

# Обновление подписей осей для каждого подграфика
fig.update_yaxes(title='Loss', row=1, col=1)
fig.update_yaxes(title='Accuracy', row=2, col=1)
fig.update_yaxes(title='Recall', row=3, col=1)
fig.update_yaxes(title='Precision', row=4, col=1)
fig.update_yaxes(title='F1', row=5, col=1)
fig.update_layout(width=1000, height=1800)

fig.show()


In [18]:
device = 'cuda'

In [19]:
# Путь для загрузки весов модели
model_save_path_cnn = os.path.join(DATASETS_PATH, 'r3d_18_weights', f"r3d_18_model_weights_{6}_epoch.pth")   # дообучаем сохраненные веса
model = VideoClassifier(num_classes=df['BD_label'].nunique())  # модель
model.load_state_dict(torch.load(model_save_path_cnn, map_location=torch.device('cuda')))  # загружаем сохраненные веса модели

model.to(device)  # Перемещаем модель на устройство


model.eval()  # режим оценки качества модели
all_preds, all_labels = [], []  # предикты
with torch.no_grad():  # на тесте не обучается
    for videos, labels in tqdm(test_loader):
        # Переносим данные и метки на то же устройство, что и модель
        videos, labels = videos.to(device), labels.to(device)

        # предсказание
        outputs = model(videos)
        preds = torch.argmax(outputs, dim=1)  # наиболее вероятный класс

        all_preds.extend(preds.cpu().numpy())   # все предикты
        all_labels.extend(labels.cpu().numpy())  # все реальные лейблы
label_names = [str(i) for i in range(df['BD_label'].nunique())]  # Список всех классов

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:01<00:00, 88.0MB/s]
100%|██████████| 312/312 [15:38<00:00,  3.01s/it]


In [20]:
le.classes_

array(['Повороты_Арабеск', 'Повороты_Аттитюд',
       'Повороты_Боковой шпагат без помощи, туловище в горизонтальном положении',
       'Повороты_Боковой шпагат с помощью',
       'Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении',
       'Повороты_В шпагате с помощью с наклоном вперед',
       'Повороты_В шпагате с помощью с наклоном назад',
       'Повороты_Вперед: свободная нога в горизонтальном положении',
       'Повороты_Задний шпагат без помощи, туловище горизонтально',
       'Повороты_Задний шпагат без помощи, туловище горизонтально в кольцо',
       'Повороты_Задний шпагат с помощью',
       'Повороты_Кольцо без помощи аттитюд с прогибом назад',
       'Повороты_Кольцо с помощью с ногой на плече',
       'Повороты_На животе груди, ноги в положении подбива с помощью («Ашрам»)',
       'Повороты_На животе груди, ноги в положении шпагата («Канаева»)',
       'Повороты_Пассе', 'Повороты_Передний шпагат без помощи',
       'Повороты_Передний шпагат с

In [26]:
# Преобразование чисел обратно в текст
all_labels_text = le.inverse_transform(all_labels)
all_preds_text = le.inverse_transform(all_preds)

# Accuracy
accuracy = accuracy_score(all_labels_text, all_preds_text)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Матрица ошибок с текстовыми метками
class_names = le.classes_
conf_matrix = confusion_matrix(all_labels_text, all_preds_text, labels=class_names)

# Тепловая карта матрицы ошибок
fig = go.Figure(data=go.Heatmap(
    z=conf_matrix,
    x=class_names,
    y=class_names,
    colorscale='Turbo',
    colorbar=dict(title="Количество предсказаний"),
    text=conf_matrix,
    texttemplate="%{text}",
    hovertemplate="Истинный: %{y}<br>Предсказанный: %{x}<br>Количество: %{z}<extra></extra>"
))

fig.update_layout(
    title="Матрица ошибок",
    xaxis_title="Предсказанный класс",
    yaxis_title="Истинный класс",
    autosize=True,
    width=2800,
    height=2200,
    paper_bgcolor='black',
    plot_bgcolor='black',
    font=dict(color='white'),
    xaxis=dict(tickangle=-45),
    yaxis=dict(autorange="reversed")
)
fig.show()


# classification_report в таблицу с текстовыми метками
report_dict = classification_report(all_labels_text, all_preds_text, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report_dict).transpose().round(2).reset_index()
report_df = report_df.rename(columns={'index': 'Класс'})

# Таблица отчёта
fig_report = go.Figure(data=[go.Table(
    header=dict(
        values=list(report_df.columns),
        fill_color='darkslategray',
        font=dict(color='white', size=12),
        align='left'
    ),
    cells=dict(
        values=[report_df[col] for col in report_df.columns],
        fill_color='black',
        font=dict(color='white'),
        align='left'
    )
)])
fig_report.update_layout(
    title="Отчёт классификации",
    paper_bgcolor='black'
)
fig_report.show()


Test Accuracy: 87.08%


In [37]:
report_df[report_df['f1-score'] == report_df['f1-score'].min()]

,Класс,precision,recall,f1-score,support
74,"Равновесия_Передний шпагат без помощи, наклон туловища назад ниже горизонтали",0.518519,0.875,0.651163,16.0


In [38]:
report_df[report_df['precision'] == report_df['precision'].min()]

,Класс,precision,recall,f1-score,support
10,Повороты_Задний шпагат с помощью,0.484848,1.0,0.653061,16.0


In [39]:
report_df[report_df['recall'] == report_df['recall'].min()]

,Класс,precision,recall,f1-score,support
57,Равновесия_Арабеск,1.0,0.5,0.666667,10.0


In [35]:
# Пример: получаем словарь из classification_report
report_dict = classification_report(all_labels_text, all_preds_text, target_names=class_names, output_dict=True)

# Преобразуем в DataFrame
report_df = pd.DataFrame(report_dict).transpose().reset_index().rename(columns={'index': 'Класс'})

# Убираем агрегированные строки (accuracy, avg)
plot_df = report_df[~report_df['Класс'].isin(['accuracy', 'macro avg', 'weighted avg'])]

# Строим график
fig = px.scatter(
    plot_df,
    x='support',
    y='f1-score',
    # text='Класс',
    title='Зависимость F1-score от Support',
    labels={'support': 'Количество примеров (support)', 'f1-score': 'F1-score'}
)
fig.update_traces(textposition='top center')
fig.update_layout(template='plotly_dark')
fig.show()

In [ ]:
# зависимость precision от support

In [ ]:
# Получаем HTML-код каждого графика
html_conf_matrix = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')
html_report = pio.to_html(fig_report, full_html=False, include_plotlyjs=False)

# Объединяем все графики в один HTML-документ
full_html = f"""
<html>
<head>
    <meta charset="utf-8" />
    <title>Все графики</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body style="background-color: black; color: white;">
    <h1>Матрица ошибок</h1>
    {html_conf_matrix}
    <h1>Отчёт классификации</h1>
    {html_report}
</body>
</html>
"""

# Сохраняем файл
with open("all_plots.html", "w", encoding="utf-8") as f:
    f.write(full_html)

print("Все графики сохранены в all_plots.html")


Все графики сохранены в all_plots.html
